**SA402 &#x25aa; Dynamic and Stochastic Models &#x25aa; Fall 2022**

# Poisson processes for fun and profit

## A really important question

* Do BTS's tweets follow a Poisson process?

## How do we determine if an arrival process is Poisson?

* One approach: __look at the interarrival times__.
    * Are the interarrival times exponentially distributed?
    * Are the interarrival times independent?

## Preliminaries

* First, we need to import a whole bunch of libraries, including [Tweepy](http://www.tweepy.org), which allows us to interface with Twitter programmatically using Python.

In [ ]:
import tweepy
import pandas as pd
import altair as alt
import math
import scipy.stats as stats
import IPython.display

* Next, we need to authenticate into Twitter.

In [ ]:
bearer_token = 'BEARER_TOKEN'

In [ ]:
client = tweepy.Client(bearer_token)

## Getting someone's tweets

* Let's first test Tweepy by grabbing some basic information about the Twitter user we want to study.

In [ ]:
username = 'BTS_twt'

In [ ]:
response = client.get_user(username=username, user_fields=['profile_image_url'])
user = response.data

print(f'Name: {user.username}')
IPython.display.Image(user.profile_image_url)

* Now, let's grab this user's last 200 tweets.

In [ ]:
n_tweets = 200

In [ ]:
public_tweets = []
for tweet in tweepy.Paginator(
    client.get_users_tweets, user.id, 
    max_results=100, tweet_fields=['created_at']
).flatten(limit=n_tweets):
    public_tweets.append(tweet)

* Next, we'll put the arrival time and text of each tweet into a __Pandas DataFrame__, which will let us analyze our data more easily.
    - [**Pandas**](https://pandas.pydata.org/) is a Python library for data manipulation and analysis.
    - You can think of a __DataFrame__ as a two-dimensional table, with rows and columns.

In [ ]:
public_tweets_raw_df = pd.DataFrame.from_records(
    [[tweet.created_at, tweet.text] for tweet in public_tweets],
    columns=['arrival_time', 'text']
)

* Just to make sure we're doing this right &mdash; let's examine some of this user's tweets by viewing the first 5 rows of this DataFrame:

In [ ]:
public_tweets_raw_df

## Computing the observed interarrival times

* Let's treat each tweet as an *observation*.


* We can compute the observed interarrival times by:
    - sorting the data by the arrival times in ascending order, and then 
    - computing the difference between consecutive arrival times.


* We convert the time differences to seconds, and then divide by $60 \times 60$ to obtain the time differences in hours.

In [ ]:
public_tweets_df = (
    public_tweets_raw_df
    .sort_values('arrival_time', ascending=True)
    .assign(
        interarrival_time=lambda x: x['arrival_time'].diff(periods=1).dt.total_seconds() / (60 * 60)
    )
)

public_tweets_df.head()

## Creating a histogram of the observed interarrival times in our data

* Now that we have the observed interarrival times, let's visualize them with a histogram.


* We will use [**Altair**](https://altair-viz.github.io/), a modern Python visualization library.


* First, a little setup:
    * We specify the number of bins we want in our histogram.
    * Based on the maximum observed interarrival time in our data, we can then compute the width of each bin.

In [ ]:
n_bins = 20

In [ ]:
max_interarrival_time = public_tweets_df['interarrival_time'].max()
print(f'Maximum interarrival time: {max_interarrival_time}')

In [ ]:
bin_width = math.ceil(max_interarrival_time / n_bins)
print(f'Bin width: {bin_width}')

In [ ]:
histogram = alt.Chart(public_tweets_df).mark_bar().encode(
    alt.X('interarrival_time:Q', 
          bin=alt.BinParams(step=bin_width),
          title='Interarrival time'),
    alt.Y('count():Q', title='Count')
)

histogram

## Fitting the observed interarrival times to an exponential distribution

* We want to see if the observed interarrival times fit an exponential distribution. 

* Recall that the exponential distribution has a parameter $\lambda$, which would correspond to the arrival rate of a Poisson process.

* What should we use for $\lambda$?

* It turns out that the __maximum likelihood estimator (MLE)__ is

$$
\hat{\lambda} = \frac{1}{\text{sample mean of the observed interarrival times}}
$$

* Let's compute this next.

In [ ]:
sample_mean_interarrival_time = public_tweets_df['interarrival_time'].mean()
print(f'Sample mean of observed interarrival times: {sample_mean_interarrival_time:.4f} hours per tweet')

In [ ]:
estimated_arrival_rate = 1 / sample_mean_interarrival_time
print(f'Estimated arrival rate: {estimated_arrival_rate:.4f} tweets per hour')

## Plotting the pdf of the MLE exponential distribution

* Let's plot the pdf of the MLE exponential distribution &mdash; the exponential distribution with parameter $\hat{\lambda}$ we found above.


* First, we need to create a DataFrame with the values of the MLE exponential distribution pdf. 

In [ ]:
n_values = 1000

interarrival_time_values = [i * max_interarrival_time / n_values for i in range(n_values)]
density_values = [stats.expon.pdf(x, scale=1 / estimated_arrival_rate) for x in interarrival_time_values]

exponential_density_df = pd.DataFrame(
    {
        'interarrival_time': interarrival_time_values,
        'density': density_values
    }
)

* Let's check our work by examining the first 5 rows of the DataFrame:

In [ ]:
exponential_density_df.head()

* Now we can use Altair to plot the pdf of the MLE exponential distribution.
    * We need to scale the pdf values to match the area under the histogram of observed interarrival times.
    * The area under the (unscaled) pdf is 1. 
    * The area under the histogram is `n_tweets * bin_width`. (Why?)

In [ ]:
pdf = alt.Chart(exponential_density_df).transform_calculate(
    scaled_density=f'datum.density * {n_tweets} * {bin_width}'
).mark_line(color='red').encode(
    alt.X('interarrival_time:Q', title='Interarrival time'),
    alt.Y('scaled_density:Q', title='Density')
)

pdf

## Comparing the histogram of observed interarrival times and the MLE exponential pdf

* To more easily compare the histogram of observed interarrival times and the MLE exponential pdf, we can simply add them together in Altair:

In [ ]:
histogram + pdf

🤔 __What do you think?__  __Are the interarrival times exponentially distributed?__

* Ideally, we would perform some goodness-of-fit tests to statistically determine whether the exponential distribution is a good fit for the interarrival times. 
    - This is covered in _SA421 Simulation Modeling_.

## Checking for independence of the interarrival times

* We also need to check independence of the interarrival times. One easy visual test is to plot the interarrival times as a time series:

In [ ]:
alt.Chart(public_tweets_df).mark_line().encode(
    alt.X('arrival_time:O', title='Arrival time'),
    alt.Y('interarrival_time:Q', title='Interarrival time')
)

🤔 __What do you think? Are the interarrival times independent?__

🤔 __What type of user would you expect to tweet according to a Poisson process?__

## #poisson?

* We can also do the same thing with hashtags. Let's grab the last 200 tweets with a certain hashtag.

In [ ]:
search_text = "#gonavy"

In [ ]:
n_tweets = 200

In [ ]:
hashtag_tweets = []
for tweet in tweepy.Paginator(
    client.search_recent_tweets, search_text, 
    max_results=100, tweet_fields=['created_at']
).flatten(limit=n_tweets):
    hashtag_tweets.append(tweet)

In [ ]:
hashtag_tweets_df = (
    pd.DataFrame.from_records(
        [[tweet.created_at, tweet.text] for tweet in hashtag_tweets],
        columns=['arrival_time', 'text']
    )
    .sort_values('arrival_time', ascending=True)
    .assign(
        interarrival_time=lambda x: x['arrival_time'].diff(periods=1).dt.total_seconds() / 60
    )
)

hashtag_tweets_df.head()

* Now, we can go through the same process as we did above:

In [ ]:
max_interarrival_time = hashtag_tweets_df['interarrival_time'].max()
sample_mean_interarrival_time = hashtag_tweets_df['interarrival_time'].mean()
estimated_arrival_rate = 1 / sample_mean_interarrival_time

n_values = 1000
exponential_density_df = pd.DataFrame(
    {
        'interarrival_time': [i * max_interarrival_time / n_values for i in range(n_values)],
        'density': [stats.expon.pdf(x, scale=1 / estimated_arrival_rate) for x in interarrival_time_values]
    }
)

n_bins = 20
bin_width = math.ceil(max_interarrival_time / n_bins)

histogram = alt.Chart(hashtag_tweets_df).mark_bar().encode(
    alt.X('interarrival_time:Q', 
          bin=alt.BinParams(step=bin_width),
          title='Interarrival time'),
    alt.Y('count():Q', title='Count')
)

pdf = alt.Chart(exponential_density_df).transform_calculate(
    scaled_density=f'datum.density * {n_tweets} * {bin_width}'
).mark_line(color='red').encode(
    alt.X('interarrival_time:Q', title='Interarrival time'),
    alt.Y('scaled_density:Q', title='Density')
)

histogram + pdf

🤔 __What do you think? Are the interarrival times from an exponential distribution?__

In [ ]:
alt.Chart(hashtag_tweets_df).mark_line().encode(
    alt.X('arrival_time:O', title='Arival time'),
    alt.Y('interarrival_time:Q', title='Interarrrival time')
)

🤔 __Are the interarrival times independent?__

🤔 __What type of hashtag would you expect to follow a Poisson process?__

## Shameless plug 😬

* If you'd like to learn how to
    - visualize data with Altair, and
    - wrangle data with Pandas,
  
  just like in this lesson, sign up for _SA463A Data Wrangling and Visualization_!